Now get an instance of the flows client.  You will be asked to validate credentials with the Globus service.  

In [10]:
import os
import time

import globus_sdk

from utils import get_specific_flow_client
# fc = get_flows_client()

Next, create a flow.  You will again be asked to validate credentials with the globus service.

## Run the Flow

Now we need to create a set of inputs for the flow, that will follow the json structure below.  The key elements that are needed are:
1. The Globus compute endpoint id
2. The Globus compute function id
3. Inputs to the Globus compute function
4. The endpoint and path from which to transfer data
5. The endpoint and path to which to transfer data

The user should choose a destination path and endpoint that they have access to.  As a sample endpoint, below the ALCF /home space is used.  To use this endpoint, the user should set a path in their home space (but remove the leading `/home` so that it will appear like `/csimpson/als_example`).

In [8]:
# lcfhome_transfer_endpoint_id = "9032dd3a-e841-4687-a163-2720da731b5b"
nersc_transfer_endpoint_id = "6bdc7956-fc0f-4ad2-989c-7aa5ee643a79"
destination_path_on_alcfhome = "/global/cfs/cdirs/als/data_mover/iribeta/" # Note that paths for transfers on the home endpoint should remove the leading /home
reconstruction_func = "91ce59ba-cbf1-45e3-b4f3-c86aeaacda42"
polaris_endpoint_id = "b33b3f5f-eb14-4bbd-89ed-9333a6ece6b6"

The example problem is setup in a directory on the Eagle filesystem at ALCF, so that endpoint and path is used as the source of the data.  All the endpoints and functions are added to a `flow_input`.

In [9]:
eagle_transfer_endpoint_id = "05d2c76a-e867-4f67-aa57-76edeb0beda0"
source_path_on_eagle = "/IRIBeta/als/example" # Note that paths for transfers on the eagle endpoint should remove the leading /eagle

function_inputs = {"rundir": "/eagle/IRIBeta/als/example"}

flow_input = {
    "input": {
      "source": {
        "id": eagle_transfer_endpoint_id,
        "path": source_path_on_eagle
      },
      "destination": {
        "id": nersc_transfer_endpoint_id,
        "path": destination_path_on_alcfhome
      },
      "recursive_tx": True,
      "compute_endpoint_id": polaris_endpoint_id,
      "compute_function_id": reconstruction_func,
      "compute_function_kwargs": function_inputs
    }
}

In [11]:
# collection_ids should contain all the transfer endpoint ids involved in the flow
collection_ids = [flow_input["input"]["source"]["id"], flow_input["input"]["destination"]["id"]]

run_client = get_specific_flow_client(reconstruction_func, collection_ids=collection_ids)

In [12]:
flow_action = run_client.run_flow(flow_input, label="ALS run", tags=["demo", "als", "tomopy"])
flow_run_id = flow_action['action_id']

print(f'Flow action started with id: {flow_run_id}')

print(f"Monitor your flow here: https://app.globus.org/runs/{flow_run_id}")
import time
flow_status = flow_action['status']
while flow_status in ['ACTIVE', 'INACTIVE']:
    time.sleep(10)
    flow_action = fc.get_run(flow_run_id)
    flow_status = flow_action['status']
    print(f'Flow status: {flow_status}')
print(f'Final status: {flow_status}')

Flow action started with id: a618a5a2-e5ff-4339-9ec7-260abc98d289
Monitor your flow here: https://app.globus.org/runs/a618a5a2-e5ff-4339-9ec7-260abc98d289
Flow status: FAILED
Final status: FAILED
